## Web Crawling
- 웹 페이지에서 데이터를 수집하는 방법에 대해서 학습

### 웹크롤링 방법

#### 웹페이지의 종류
- 정적인 페이지 : 웹 브라우져에 화면이 한번 뜨면 이벤트에 의한 화면의 변경이 없는 페이지
- 동적인 페이지 : 웹 브라우져에 화면이 뜨고 이벤트가 발생하면 서버에서 데이터를 가져와 화면을 변경하는 페이지

#### requests 이용
- 받아오는 문자열에 따라 두가지 방법으로 구분
    - json 문자열로 받아서 파싱하는 방법 : 주로 동적 페이지 크롤링할때 사용
    - html 문자열로 받아서 파싱하는 방법 : 주로 정적 페이지 크롤링할때 사용
        
#### selenium 이용
- 브라우져를 직접 열어서 데이터를 받는 방법

#### 크롤링 방법에 따른 속도
- requests json > requests html > selenium

### Crawling Naver Stock Data
- 네이버 증권 사이트에서 주가 데이터 수집
- 수집할 데이터 : 일별 kospi, kosdaq 주가, 일별 환율(exchange rate) 데이터
- 데이터 수집 절차
    - 웹서비스 분석 : url
    - 서버에 데이터 요청 : request(url) > response : json(str)
    - 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [1]:
import warnings
warnings.filterwarnings('ignore') # 경고 문구 출력 X

# 라이브러리 볼러오기
import pandas as pd
import requests # url 이용해서 데이터 받아오는 라이브러리

#### 1. 웹서비스 분석 : url
- pc 웹페이지가 복잡하면 mobile 웹페이지에서 수집

In [24]:
page_size, page = 60, 1

url = f'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={page_size}&page={page}'

#### 2. 서버에 데이터 요청 : request(url) > response : json(str)
- response의 status code가 200이 나오는지 확인
- 403이나 500이 나오면 request가 잘못되거나 web server에서 수집이 안되도록 설정이 된것임
    - header 설정 또는 selenium 사용
- 200이 나오더라도 response 안에 있는 내용을 확인 > 확인하는 방법 : response.text

In [4]:
help(requests.get)

Help on function get in module requests.api:

get(url, params=None, **kwargs)
    Sends a GET request.
    
    :param url: URL for the new :class:`Request` object.
    :param params: (optional) Dictionary, list of tuples or bytes to send
        in the query string for the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



In [25]:
response = requests.get(url)
dir(response)
response.text[:50] # datatype : str

'[{"localTradedAt":"2024-03-07","closePrice":"2,646'

#### 3. 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [27]:
help(response.json)
data = response.json() # str -> list, dict
df = pd.DataFrame(data) # list, dict -> DataFrame
df.head()

Help on method json in module requests.models:

json(**kwargs) method of requests.models.Response instance
    Returns the json-encoded content of a response, if any.
    
    :param \*\*kwargs: Optional arguments that ``json.loads`` takes.
    :raises requests.exceptions.JSONDecodeError: If the response body does not
        contain valid json.



,localTradedAt,closePrice,compareToPreviousClosePrice,compareToPreviousPrice,fluctuationsRatio,openPrice,highPrice,lowPrice
0,2024-03-07,"2,646.32",4.83,"{'code': '2', 'text': '상승', 'name': 'RISING'}",0.18,"2,653.98","2,660.26","2,633.57"
1,2024-03-06,"2,641.49",-7.91,"{'code': '5', 'text': '하락', 'name': 'FALLING'}",-0.30,"2,638.84","2,649.78","2,630.16"
2,2024-03-05,"2,649.40",-24.87,"{'code': '5', 'text': '하락', 'name': 'FALLING'}",-0.93,"2,660.80","2,684.83","2,649.35"
3,2024-03-04,"2,674.27",31.91,"{'code': '2', 'text': '상승', 'name': 'RISING'}",1.21,"2,664.52","2,682.80","2,662.32"
4,2024-02-29,"2,642.36",-9.93,"{'code': '5', 'text': '하락', 'name': 'FALLING'}",-0.37,"2,643.48","2,647.56","2,628.62"


#### 4. 함수로 만들기

In [34]:
def stock_price(code = 'KOSPI', page_size=60, page=1):
    # 웹 서비스 분석
    url = f'https://m.stock.naver.com/api/index/{code}/price?pageSize={page_size}&page={page}'
    # request -> response : json
    response = requests.get(url)
    # json(str) -> list, dict -> DataFrame
    data = response.json()
    df = pd.DataFrame(data)
    
    return df[['localTradedAt', 'closePrice']]

In [35]:
df = stock_price(code = 'KOSDAQ')
df.head()

,localTradedAt,closePrice
0,2024-03-07,863.37
1,2024-03-06,870.67
2,2024-03-05,866.37
3,2024-03-04,872.97
4,2024-02-29,862.96


#### 5. 원달러 환율 데이터 수집 : 실습

In [55]:
def dollar_rate(page=1, page_size=60):
    # url
    url = f'https://m.stock.naver.com/front-api/v1/marketIndex/prices?pageSize={page_size}&category=exchange&reutersCode=FX_USDKRW&page={page}'
    # requests
    response = requests.get(url)
    # json -> dict
    data =  response.json()['result']
 
    df = pd.DataFrame(data)
    return df

In [57]:
dollar_rate().head()

,localTradedAt,closePrice,fluctuations,fluctuationsRatio,fluctuationsType,cashBuyValue,cashSellValue,sendValue,receiveValue
0,2024-03-07,"1,331.50",-2.00,-0.15,"{'code': '5', 'text': '하락', 'name': 'FALLING'}","1,354.8","1,308.2","1,344.5","1,318.5"
1,2024-03-06,"1,333.50",-1.50,-0.11,"{'code': '5', 'text': '하락', 'name': 'FALLING'}","1,356.83","1,310.17","1,346.5","1,320.5"
2,2024-03-05,"1,335.00",2.00,0.15,"{'code': '2', 'text': '상승', 'name': 'RISING'}","1,358.36","1,311.64","1,348.0","1,322.0"
3,2024-03-04,"1,333.00",-3.00,-0.22,"{'code': '5', 'text': '하락', 'name': 'FALLING'}","1,356.32","1,309.68","1,346.0","1,320.0"
4,2024-02-29,"1,336.00",0.00,0.00,"{'code': '3', 'text': '보합', 'name': 'UNCHANGED'}","1,359.38","1,312.62","1,349.0","1,323.0"


#### 6. 시각화

In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#### 7. 데이터 스케일링
- min max scaling


- $z = \frac{x_i - min(x)}{max(x) - min(x)} (0 \leqq z \leqq 1)$


- latex syntax : `https://jjycjnmath.tistory.com/117`

In [ ]:
from sklearn.preprocessing import minmax_scale

#### 8. 상관관계 분석
- 피어슨 상관계수(Pearson Correlation Coefficient)
- 두 데이터 집합의 상관도를 분석할때 사용되는 지표
- 상관계수의 해석
    - -1에 가까울수록 서로 반대방향으로 움직임
    - 1에 가까울수록 서로 같은방향으로 움직임
    - 0에 가까울수록 두 데이터는 관계가 없음